Trying to make a $^\circ$F version of the chart below.

<a href = "https://en.wikipedia.org/wiki/Psychrometrics#Psychrometric_charts"> <img src="https://upload.wikimedia.org/wikipedia/commons/9/9d/PsychrometricChart.SeaLevel.SI.svg" alt = "Psychrometric chart image" Title="Psychrometric Chart (SI)"  style="max-width:800px; max-height:800px; border:1px solid blue;  "/></a>


##Starting with the RH curves

In [214]:
T = range(-10,60,5)

def X(t):
    "X location in pixels based on temperature"
    x = (t+10.5712164925)/0.0869477992
    return x
X(-10)

6.569648659951357

In [95]:
#For copying the pixel data
txt="""140
144
147"""
', '.join(txt.splitlines())

'140, 144, 147'

In [215]:

# temp = [-10, -5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
rh = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100] # RH values looked at
y_max = 471 #Maximum height of the chart in pixels
# Δ = Vertical spacing between RH curves at each temperature
Δ ={-10:'2, 2, 2, 3, 2, 3, 2, 3, 2, 2',
    -5:'3, 3, 4, 3.5, 3.5, 3, 3.5, 4, 3, 4.5',
    0:'4, 5, 6.5, 5.5, 4.5, 4.5, 5.5, 5.5, 4.5, 7.5',
    5:'6, 7.5, 9.5, 7, 7, 7.5, 7.5, 9, 7, 9',
    10:'9, 10.5, 12.5, 10, 10.5, 11.5, 10.5, 12, 10.5, 13',
    15:'13, 15, 16.5, 14.5, 15, 16, 15, 16, 16, 16',
    20:'19, 21, 21, 22, 20, 21, 21, 22, 22, 23',
    25:'26, 29, 27.5, 30.5, 29, 27, 29, 30, 30, 29',
    30:'36, 39, 37, 41, 38, 38, 39, 39, 42, 42',
    35:'48, 51, 50, 54, 50, 53, 53, 53, 56',
    40:'64, 66, 67, 71, 67, 72',
    45:'83.5, 86.5, 88, 92, 91',
    50:'109, 112, 115, 120',
    55:'140, 144, 147'} 

def _d(d_txt):
    "Convert Δ text into numbers"
    d = d_txt.split(', ')
    d = [float(d_) for d_ in d]
    return d

def y(C,RH):
    "Height in pixels from the bottom of the graph at temperature C for humidity RH"
    if C not in T:
        er = "{} is not a value temperature".format(C)
        raise(ValueError(er))
    else:
        d = _d(Δ[C])
#     print(d)
    RH_ = rh[:len(d)]
#     print(RH_)
    slope, intercept, r_value, p_value, std_err = stats.linregress(RH_,d)
#     print((slope, intercept, r_value))
    
    y_ = (slope*(RH+10)+intercept*2)/20*(RH)
    if y_>y_max:
        er = "This is outside of the bound."
        raise(RuntimeError(er))
    return y_
# 501-y(40,70)

In [188]:
# txt_y_100 = '24, 36, 54, 78, 110, 154, 212, 288, 392'
# txt_y_70 = '17, 24.5, 36.5, 53, 76, 106, 146, 199, 269, 359'
# txt_y_40 = '10, 15, 21.5, 31, 43, 60, 83, 114, 154, 204, 269, 351, 457'
# txt_y_20 = '5, 7, 10, 14.5, 20.5, 29, 41, 36, 75.5, 100, 132.5, 172.5, 222, 286'
# txt_y_10 = '3, 4, 5, 7, 10, 14, 20, 27, 37, 49, 65, 85, 110, 141'
def Y(RH):
    "Generate a list of vertical locations for curve fitting"
    y_ = []
    for t in temp:
        try:
            y__ = y(t, RH)
            y_.append(y__)
        except RuntimeError:
            break
    return y_

def log_Y(RH):
    "Using a log fit"
    y = Y(RH)
    y_ = [np.log10(y_) for y_ in y]
    return y_
Y(100)

[23.0,
 35.0,
 53.0,
 77.0,
 110.00000000000001,
 153.0,
 211.99999999999997,
 287.0,
 391.0]

Assumed function of RH curves.

$$Y = 10^{a x^2 +b x +c}$$

$$Y = C 10^{a x^2 + b x}$$

In [211]:
from scipy.optimize import leastsq
from numpy import exp
import numpy as np
from scipy import stats
from collections import namedtuple
Coeff_1 = namedtuple("Coeff_1","a b")
Coeff = namedtuple("Coeff","a b c")
Coeff_3 = namedtuple("Coeff_3","a b c d")

def fit(RH):
    T_ = T[:len(Y(RH))]
    cof = np.polyfit([X(t) for t in T_], log_Y(RH) , deg=2)
    coff = Coeff(*[float(co) for co in cof])
#     print(coff)
    return coff
 
c = [fit(RH) for RH in rh]
for C_ in c:
    print(C_)

Coeff(a=-9.250512969367285e-07, b=0.0031179040587332135, c=0.3145977020986204)
Coeff(a=-9.111862315980408e-07, b=0.0031091700092078208, c=0.6204025340992709)
Coeff(a=-8.976295958044022e-07, b=0.003100636313262496, c=0.8012077723685274)
Coeff(a=-9.097805266394754e-07, b=0.0031069497362675473, c=0.929596910999797)
Coeff(a=-9.45873122872502e-07, b=0.0031246807967751345, c=1.0291755274929153)
Coeff(a=-9.88165328267249e-07, b=0.003143417064663358, c=1.1110817842765233)
Coeff(a=-1.061268614808797e-06, b=0.003173793132327919, c=1.1801476820406356)
Coeff(a=-1.0774121259150998e-06, b=0.003179225031601445, c=1.2417317212022807)
Coeff(a=-1.093327589032915e-06, b=0.0031845985065241888, c=1.2964385539158862)
Coeff(a=-1.2173977724630225e-06, b=0.003232471874166154, c=1.3434315288932808)


Coeff(a=-1.0744257747223274e-06, b=0.0031308808364933723, c=1.3604178818352384)
Coeff(a=-9.167266993577851e-07, b=0.0030540081353169716, c=1.2038119714629498)
Coeff(a=-7.490756999530651e-07, b=0.0029196207343187577, c=0.9855328916528928)
Coeff(a=-4.211367493712049e-07, b=0.002684476616864855, c=0.6817789666629894)
Coeff(a=-3.5782964394378004e-07, b=0.0025757820514518545, c=0.4252206326163717)

* $a$ seems to be a 3$^{\text{rd}}$ order function with RH
* $b$ seems to be a 3$^{\text{rd}}$ order function with RH
* $c$ seems to be a logrithmic function of RH

In [216]:
cx = [RH for RH in rh]
# np.polyfit(cx, [C.a for C in c], deg=2)
c_a = Coeff_3(*[float(co) for co in np.polyfit(cx, [C.a for C in c], deg=3)])
c_b = Coeff_3(*[float(co) for co in np.polyfit(cx, [C.b for C in c], deg=3)])
slope, intercept, r_value, p_value, std_err = stats.linregress(cx,[10**C.c for C in c])
c_c = Coeff_1(slope, intercept)

def aa_2(RH):
    "Return coefficents for 2nd order function used in the RH log curve"
    aa = c_a.a*RH**3 +c_a.b*RH**2+c_a.c*RH+c_a.d
    ab = c_b.a*RH**3 +c_b.b*RH**2+c_b.c*RH+c_b.d
    ac = np.log10(c_c.a*RH +c_c.b)
    re = Coeff(aa,ab,ac)
    print(re)
    return re

In [213]:
print(c_a)
print(c_b)
print(c_c)

Coeff_3(a=3.4332804658978364e-13, b=-1.0359530598417374e-10, c=4.563607924634942e-09, d=-9.587224623430847e-07)
Coeff_3(a=-2.5092003664316415e-10, b=6.016359515430105e-08, c=-2.657346845678339e-06, d=0.003137712198605732)
Coeff_1(a=0.22238915394137665, b=-0.32088482933889928)


##Testing fitness

$$Y = 10^{a x^2 +b x +c}$$

$$Y = C 10^{a x^2 + b x}$$

In [209]:
C = fit(10)
C = aa_2(20)
x = X(55)
501-10**(C.a*x**2+C.b*x+C.c)

Coeff(a=-9.061418018713372e-07, b=0.0031066233394607405, c=0.61562376128585528)


223.59561619139492

**It is close but can vary as much at 6 pixels, so not good enough.**

Conclusion: Use the SI version.